# Tutorial 3 – Apply KOs, Run MOMA and Integrate External Data

This exercise shows how to run FBA and minimization of metabolic adjustment (MOMA) simulations and how one can use GEMs as a scaffold for interpreting microarray data. A simplified model of yeast metabolism is used in this approach as an example.

Open tutorial3.m in MATLAB to begin this exercise. This script contains all the functions needed to complete this exercise.

The solutions to this exercise can be found in tutorial3_solutions.m file.

Figure 2 shows the system, this time it is a central carbon metabolism. Note that there are slight differences in naming of metabolites; particularly all metabolites have their compartment in their name.

![fig1](./tutorial_data/tutorial_3_fig_1.png)<br>
**Figure 2.** Central carbon metabolism in yeast (adapted from Förster, 2003).

1.	Open the file smallYeast.xlsx. One will hopefully recognize the model structure from Tutorial 2, even though some more stuff is added this time. The most important difference is the composition for metabolites and gene associations for reactions. Metabolite composition information is useful for finding errors in the GEMs and knowing gene associations allows to link the GEMs to other types of data, such as microarrays.
By default, the production of acetate, biomass, glycerol, CO2, and ethanol is allowed and the consumption of O2 and glucose is not allowed. Try to validate the model by confirming the results from Tutorial 2 (the maximal ATP production in glycolysis). One can set the simulation parameters in the Excel file like in Tutorial 2 or try to use the setParam function instead (see tutorial3.m). Just remember that sucrose was used in the first example while glucose is used here.

In [1]:
setRavenSolver('gurobi')

In [2]:
% load model
model = importExcelModel('./tutorial_data/smallYeast.xlsx');
printModelStats(model, true, true)

NOTE: DEFAULT LOWER not supplied. Uses -1000
NOTE: DEFAULT UPPER not supplied. Uses 1000
Network statistics for smallYeast: Central carbon metabolism for yeast
Genes*				61
	cytosol	52
	mitochondria	17

Reactions*			53
	cytosol	45
	mitochondria	19
Unique reactions**	53

Metabolites			52
	cytosol	35
	mitochondria	17
Unique metabolites	45

* Genes and reactions are counted for each compartment if any of the corresponding metabolites are in that compartment. The sum may therefore not add up to the total number.
** Unique reactions are defined as being biochemically unique (no compartmentalization)

Short model quality summary for smallYeast: Central carbon metabolism for yeast
Reactions which could not be elementally balanced	2
	biomassOUT
	GROWTH

Reactions which are elementally unbalanced	19
	acOUT	carbon, oxygen, hydrogen
	co2OUT	carbon, oxygen
	ethOUT	carbon, oxygen, hydrogen
	glyOUT	carbon, oxygen, hydrogen
	glcIN	carbon, oxygen, hydrogen
	o2IN	oxygen
	ethIN	carbon, oxygen, hydrogen


In [3]:
% get objective
fprintf('Current objective function: %s\n\n', strjoin(model.rxns(model.c == 1)))
% show transport reactions
fprintf('Transport reactions:\n\t- %s\n', strjoin(model.rxns(getTransportRxns(model) == 1), '\n\t- '))


Current objective function: biomassOUT

Transport reactions:
	- CAT2
	- CO2TRANS
	- GROWTH
	- ShuttleX


In [4]:
% validate ATP from glucose in aerobic conditions
validation_model= setParam(model,'obj',{'ATPX'},1);
validation_model= setParam(validation_model, "ub", {"glcIN", "o2IN"}, [1, 1000]);
validation_model_sol = solveLP(validation_model);
printFluxes(validation_model, validation_model_sol.x, false, 10^-7, [], '%rxnID (%rxnName):\n\t%eqn\n\t%flux\n', ["ATP"]);

FLUXES:
HXK (Hexokinase):
	alpha-D-glucose[c] + ATP[c] => ADP[c] + alpha-D-glucose 6-phosphate[c]
	1
PFK (Phosphofructokinase):
	ATP[c] + beta-D-fructofuranose 6-phosphate[c] => ADP[c] + beta-D-fructofuranose 1,6-bisphosphate[c]
	1
PGK (Phosphoglycerate kinase):
	3-phospho-D-glyceroyl phosphate[c] + ADP[c] <=> 3-phospho-D-glycerate[c] + ATP[c]
	2
PYK (Pyruvate kinase):
	ADP[c] + phosphoenolpyruvate[c] => ATP[c] + pyruvate[c]
	2
ATPX (ATP hydrolysis):
	ATP[c] => ADP[c] + phosphate[c]
	2


In [5]:
% depure variables that will be never used
clear validation_model validation_model_sol

2.	Calculate the maximal yields of biomass, ethanol, glycerol, and acetate on glucose and fully aerobic (biomass is in 1/h and the other fluxes are in mmol/gDW/h).

In [6]:
% set conditions
aerobic_conditions_model = setParam(model, "ub", {"glcIN", "o2IN"}, [1, 1000]);
reactions_to_evaluate = ["ethOUT", "acOUT", "glyOUT", "biomassOUT"];
reactions_to_names_map = dictionary(reactions_to_evaluate, ["ethanol" , "acetate", "glycerol", "biomass"]);
reactions_to_units_map = dictionary(reactions_to_evaluate, ["mmol/gDW/h", "mmol/gDW/h", "mmol/gDW/h", "/h"] );

In [7]:
for reaction = reactions_to_evaluate
    current_model = setParam(aerobic_conditions_model,'obj',{reaction}, 1);
    current_solution = solveLP(current_model);
    fprintf('- Maximum yield of %s is %5.4f %s.\n',...
    reactions_to_names_map(reaction), current_solution.f*-1, reactions_to_units_map(reaction));
end

- Maximum yield of ethanol is 2.0000 mmol/gDW/h.
- Maximum yield of acetate is 0.6519 mmol/gDW/h.
- Maximum yield of glycerol is 0.5470 mmol/gDW/h.
- Maximum yield of biomass is 0.1222 /h.


In [8]:
clear aerobic_conditions_model reactions_to_evaluate reactions_to_names_map reactions_to_units_map
clear reaction current_model current_solution

3.	Assume that one wants to predict how the flux distribution would change if the conditions are changed. The user should know by now how to import models and set parameters. Import the model twice and generate one model which has access to 1 mmol/gDW/h of glucose and unconstrained oxygen, and one which has access to 1 mmol/gDW/h of glucose and 0.5 mmol/gDW/h of O2. Optimize for growth under both conditions. What were the outputs? Look in the tutorial and try to run drawMap for the two models. Open the resulting PDF file and see if one can understand (in broad terms) how metabolism changes between the two conditions.

In [9]:
% set conditions
aerobic_model = setParam(model, "ub", {"glcIN", "o2IN"}, [1, 1000]);
anaerobic_model = setParam(model, "ub", {"glcIN", "o2IN"}, [1, 0.5]);
models_to_check = [aerobic_model, anaerobic_model];
messages = ["aerobic", "anaerobic"];
stored_solution_distribution = [];

In [10]:
for index = 1:2
    current_model = models_to_check(index);
    current_solution = solveLP(current_model);
    stored_solution_distribution = [stored_solution_distribution, current_solution.x];
    fprintf('- Maximum yield of biomass in %s conditions is %5.4f /h\n', messages(index), current_solution.f*-1);
end

- Maximum yield of biomass in aerobic conditions is 0.1222 /h
- Maximum yield of biomass in anaerobic conditions is 0.0746 /h


In [11]:
% Draw a map
% drawMap(title,pathway,modelA,conditionA,conditionB,modelB,filename,cutOff,supressOpen)
load("./tutorial_data/pathway.mat", "pathway");
drawMap('Aerobic vs. Anaerobic', pathway, ...
    aerobic_model, stored_solution_distribution(:, 1), ...
    stored_solution_distribution(:, 2), anaerobic_model, ...
    './output/tutorial_3_mapFBA.pdf', 10^-5);

File saved as ./output/tutorial_3_mapFBA.pdf


In [12]:
clear aerobic_model anaerobic_model models_to_check messages stored_solution_distribution
clear index current_model current_solution

- Alcohol is produced at anaerobic conditions (red).
- Acetate is produced at aerobic onditions (green).

This makes sense since the alcohol group (R-OH) is more reduced than the carboxilyc group (R-COOH). In anaerobic conditions will be more favored fermentation.


![alcohol production (anaerobic conditions) vs acetate production (aerobic condition)](./output/tutorial_3_answer_point3.png)

4.	Say that the goal is to increase glycerol production in the strain (e.g., to make sweeter wine). FBA is useful for suggesting gene deletions which can couple the set goal with the overall objective for the cell, which is to grow as fast and efficient as possible. Set the parameters to maximize for growth under anaerobic conditions and run the code for making a single gene deletion using FBA. How large was the difference in glycerol production? Can one come up with a hypothesis as to how the deletion leads to higher glycerol production? One can also use the function followChanged, which compares two flux distributions and let the user search between them (see tutorial3.m)

- [findGeneDeletions](https://sysbiochalmers.github.io/RAVEN/doc/core/findGeneDeletions.html)

```MATLAB
function [genes, fluxes, originalGenes, details, grRatioMuts]=findGeneDeletions(model,testType,analysisType,refModel,oeFactor)
```
Deletes genes, optimizes the model, and keeps track of the resulting fluxes. This is used for identifying gene deletion targets.

In [13]:
anaerobicModel = setParam(model, "ub", {"glcIN", "o2IN"}, [1, 0]);
[genes, fluxes, originalGenes, details, grRatioMuts] = findGeneDeletions(anaerobicModel, 'sgd', 'fba');


- [getIndexes](https://sysbiochalmers.github.io/RAVEN/doc/core/getIndexes.html)

```MATLAB
function indexes=getIndexes(model, objects, type, returnLogical)
```
Retrieves the indexes for a list of reactions or metabolite.

In [14]:
indexes = getIndexes(anaerobicModel,{'biomassOUT', 'glyOUT'},'rxns');
biomassOutIndex = indexes(1);
glycerolOutIndex = indexes(2);


- [find](https://www.mathworks.com/help/matlab/ref/find.html)

```MATLAB
[row,col] = find(___)
```

Returns the row and column subscripts of each nonzero element in array X using any of the input arguments in previous syntaxes.

- [max](https://www.mathworks.com/help/matlab/ref/max.html)

```MATLAB
[M,I] = max(___)
```

Also returns the index into the operating dimension that corresponds to the first occurrence of the maximum value of A.

In [15]:
% set the threshold from which the value is considered equal to zero
tolerance = 10^-7;
% "fluxes" is a matrix of size {number of ractions} x {deleted genes}
% get biomassOut fluxes for all deletions and find those with growth
columnsWithGrow = find(fluxes(biomassOutIndex, :) >= tolerance);
% get glycerolOut fluxes for the choosen deletions
glycerolFluxes = fluxes(glycerolOutIndex, columnsWithGrow);
% get the maximum possible glycerol production
maximumGlycerolProduction = max(glycerolFluxes);
% First, remember that more than one deletion can give the maximum glycerol production.
% Second, the apropiate way to compare float point values is using a delta. Do not use the "==" operator!
columnsWithMaximumGrow = columnsWithGrow(abs(glycerolFluxes - maximumGlycerolProduction) < tolerance);
targetGenes = originalGenes(columnsWithMaximumGrow);
fprintf('The maximum glycerol production is: %5.4f mmol/gDW/h.\nRelated gene single deletions:\n\t- %s',... 
    full(maximumGlycerolProduction), strjoin(targetGenes, '\n\t- '));

The maximum glycerol production is: 0.3701 mmol/gDW/h.
Related gene single deletions:
	- YLR354C
	- YNL241C

In [16]:
% related reactions
targetReactions = anaerobicModel.rxns(contains(anaerobicModel.grRules, targetGenes));
targetReactionNames = anaerobicModel.rxnNames(contains(anaerobicModel.grRules, targetGenes));
disp(table(targetGenes, targetReactions, targetReactionNames));

    targetGenes    targetReactions              targetReactionNames          
    ___________    _______________    _______________________________________

    {'YLR354C'}       {'ZWF' }        {'Glucose-6-phosphate 1-dehydrogenase'}
    {'YNL241C'}       {'TAL1'}        {'Transaldolase'                      }



In [17]:
wildTypeFluxes = solveLP(anaerobicModel).x;
electronCarriers = {'NADPH' 'NADH' 'NAD(+)' 'NADP(+)'};
% Notice that it is the transpose of  targetReactions. MATLAB loops go over the columns
for reaction = targetReactions'
    fprintf('Current reaction: %s\n', string(reaction));
    % set condtions
    mutatedModel =setParam(anaerobicModel,'eq', reaction, 0);
    mutatedFluxes = solveLP(mutatedModel).x;
    % export file
    title = reaction + " deletion vs. wildtype" ;
    fileName = strjoin(['./output/tutorial_3_map', reaction, '.pdf'], '');
    drawMap(title , pathway, ...
    mutatedModel, mutatedFluxes, ...
    wildTypeFluxes, anaerobicModel, ...
    fileName, 10^-5);
    % print some relevant changes related with electrons carriers (reduction power)
    followChanged(anaerobicModel, mutatedFluxes, wildTypeFluxes,...
    20, 10^-2, 0, electronCarriers);
    fprintf('-----------------------------------------------\n')  ;
end


Current reaction: ZWF
File saved as ./output/tutorial_3_mapZWF.pdf
These reactions have flux values that differ by more than 20 percent, absolute values above 0.01, and a total difference above 0 (4 reactions)

Only prints reactions involving one or more of the following metabolites:
NADPH NADH NAD(+) NADP(+) 

ZWF: alpha-D-glucose 6-phosphate[c] + NADP(+)[c] => 6-O-phosphono-D-glucono-1,5-lactone[c] + NADPH[c]
	Glucose-6-phosphate 1-dehydrogenase
	Flux: 0 Reference flux: 0.10432 Difference: -0.10432

GND: 6-phospho-D-gluconate[c] + NADP(+)[c] => CO2[c] + D-ribulose 5-phosphate[c] + NADPH[c]
	6-phosphogluconate dehydrogenase, decarboxylating
	Flux: 0 Reference flux: 0.10432 Difference: -0.10432

DAR: glycerone phosphate[c] + NADH[c] => glycerol monophosphate[c] + NAD(+)[c]
	glycerol-3-phosphate dehydrogenase
	Flux: 0.37288 Reference flux: 0.23556 Difference: 0.13732

ALD6: acetaldehyde[c] + NADP(+)[c] => acetate[c] + NADPH[c]
	Aldehyde dehydrogenase
	Flux: 0.25424 Reference flux: 0.094

- How large was the difference in glycerol production?

In [18]:
glycerolProductionIndex = getIndexes(anaerobicModel, 'glyOUT', 'rxns');
fprintf('Glycerol production:\n');
disp(table(mutatedFluxes(glycerolProductionIndex), ...
    wildTypeFluxes(glycerolProductionIndex), ...
    (mutatedFluxes(glycerolProductionIndex)/wildTypeFluxes(glycerolProductionIndex) - 1) * 100, ...
    'VariableNames', {'mutant', 'wild-type', 'relative difference mutant vs. WT (%)'}));

Glycerol production:
    mutant     wild-type    relative difference mutant vs. WT (%)
    _______    _________    _____________________________________

    0.37006     0.23219                    59.375                



- Can one come up with a hypothesis as to how the deletion leads to higher glycerol production?

![answer4](./output/tutorial_3_answer_point4.png)

Knock out of glucose-6-phosphate 1-dehydrogenase ([zwf](https://www.kegg.jp/entry/K00036)) increase the Glucose-6P avaiable for conversión to Fructose-6P and Glycerone-P. [KEGG map](https://www.kegg.jp/pathway/sce01200+YNL241C).

In [19]:
clear anaerobicModel fileName maximumGlycerolProduction fluxes reaction
clear wildTypeFluxes genes modelMutatedZWF referenceFluxes biomassOutIndex
clear glycerolFluxes targetGenes columnsWithGrow glycerolOutIndex mutatedFluxes
clear targetReactionNames columnsWithMaximumGrow grRatioMuts mutatedModel
clear details indexes originalGenes title targetReactions glycerolProductionIndex

5.	FBA assumes that cells have the same objective even after a perturbation; to grow as efficiently as possible. This is probably true after long evolution times, but shortly after, say, a gene deletion the cell might have other objectives. MOMA is based on that the perturbed cell would like to change its metabolism as little as possible. MOMA is normally used when one has the experimental data for the wild type of strain and would like to guess how the phenotype would change after a perturbation. These are some batch data for growth on glucose:

    - acOUT (Production of acetate): 0<br>
    - biomassOUT (Production of biomass): 0.67706<br>
    - co2OUT (Production of CO2): 22.4122<br>
    - ethOUT (Production of ethanol): 19.0946<br>
    - glyOUT (Production of glycerol): 1.4717<br>
    - glcIN (Uptake of glucose): 15<br>
    - o2IN (Uptake of O2): 1.6<br>
    - ethIN (Uptake of ethanol): 0<br>

Load one model with these constraints and one without any constraints on the exchange reactions but with the ZWF reaction turned off. Run the qMOMA function and plot the flux distributions. Is the glycerol production larger?

In [20]:
%%%%%%%%%%%%% set up simulation %%%%%%%%%%%%%
experimentalFluxNames = {"acOUT", "biomassOUT", "co2OUT", "ethOUT", "glyOUT", "glcIN", "o2IN", "ethIN"};
experimentalFluxValues = [0, 0.67706, 22.4122, 19.0946, 1.4717, 15, 1.6, 0];

In [21]:
% With this data the model is infeasible, so we need to relax a bit the constraints
wildTypeModel = setParam(model, "eq", experimentalFluxNames, experimentalFluxValues);
fprintf('wildTypeModel optimization message:\n\t%s', solveLP(wildTypeModel).msg);

wildTypeModel optimization message:
	The problem is infeasible

In [22]:
wildTypeModel = setParam(model, "lb", experimentalFluxNames, experimentalFluxValues * 0.99);
wildTypeModel = setParam(wildTypeModel, "ub", experimentalFluxNames, experimentalFluxValues * 1.01);
fprintf('wildTypeModel optimization message:\n\t%s', solveLP(wildTypeModel).msg);

wildTypeModel optimization message:
	Optimal solution found

In [23]:
% now set up the mutant model
mutantModel = setParam(model, 'eq', {'ZWF'}, 0);
mutantModel = setParam(mutantModel, "lb", experimentalFluxNames, 0);
mutantModel = setParam(mutantModel, "ub", experimentalFluxNames, 1000);

[qMOMA](https://sysbiochalmers.github.io/RAVEN/doc/solver/qMOMA.html)

```MATLAB
function [fluxA,fluxB, flag]=qMOMA(modelA,modelB,fluxMinWeight)
```

Uses quadratic programming to minimize the sum((fluxAi - fluxBi)^2)


❗ Warning: Seems like it is neccesary a MATLAB add-on:

```shell
'quadprog' requires Optimization Toolbox.

Error in qMOMA (line 74)
x=quadprog(H,zeros(size(H,1),1),[],[],fullS,fullB,fullLB,fullUB);
```
I am not sure why this add-on is needed if the enviroment already have a working solver (Gurobi) 😕

In [24]:
% IMPORTANT! remember that modelB is the reference case
[mutantFluxes, wildTypeFluxes, flag] = qMOMA(mutantModel, wildTypeModel);


Minimum found that satisfies the constraints.

Optimization completed because the objective function is non-decreasing in 
feasible directions, to within the value of the optimality tolerance,
and constraints are satisfied to within the value of the constraint tolerance.



In [25]:
followChanged(wildTypeModel, mutantFluxes, wildTypeFluxes, 10, 10^-2, 10^-8,  electronCarriers);

These reactions have flux values that differ by more than 10 percent, absolute values above 0.01, and a total difference above 1e-08 (10 reactions)

Only prints reactions involving one or more of the following metabolites:
NADPH NADH NAD(+) NADP(+) 

ZWF: alpha-D-glucose 6-phosphate[c] + NADP(+)[c] => 6-O-phosphono-D-glucono-1,5-lactone[c] + NADPH[c]
	Glucose-6-phosphate 1-dehydrogenase
	Flux: 0 Reference flux: 2.0686 Difference: -2.0686

GND: 6-phospho-D-gluconate[c] + NADP(+)[c] => CO2[c] + D-ribulose 5-phosphate[c] + NADPH[c]
	6-phosphogluconate dehydrogenase, decarboxylating
	Flux: 0 Reference flux: 2.0686 Difference: -2.0686

DAR: glycerone phosphate[c] + NADH[c] => glycerol monophosphate[c] + NAD(+)[c]
	glycerol-3-phosphate dehydrogenase
	Flux: 3.0085 Reference flux: 1.5534 Difference: 1.455

ADH1: acetaldehyde[c] + NADH[c] <=> ethanol[c] + NAD(+)[c]
	Alcohol dehydrogenase
	Flux: 17.209 Reference flux: 19.1521 Difference: -1.9431

ALD6: acetaldehyde[c] + NADP(+)[c] => acetate[c] 

In [26]:
drawMap('ZWF deletion vs wild type', pathway, ...
    wildTypeModel, mutantFluxes, ...
    wildTypeFluxes, mutantModel,'./output/tutorial_3_mapMOMA.pdf',10^-5);

File saved as ./output/tutorial_3_mapMOMA.pdf


- Is the glycerol production larger?

In [27]:
glycerolProductionIndex = getIndexes(wildTypeModel, 'glyOUT', 'rxns');
fprintf('Glycerol production:\n')
disp(table(mutantFluxes(glycerolProductionIndex), ...
    wildTypeFluxes(glycerolProductionIndex), ...
        mutantFluxes(glycerolProductionIndex)/wildTypeFluxes(glycerolProductionIndex) , ...
    'VariableNames', {'mutant', 'wild-type', 'mutant/wildType'}))

Glycerol production:
    mutant    wild-type    mutant/wildType
    ______    _________    _______________

    2.9621     1.4864          1.9928     



In [28]:
clear flag experimentalFluxNames experimentalFluxValues glycerolProductionIndex
clear wildTypeModel mutantModel wildTypeFluxes mutantFluxes electronCarriers

6.	GEMs can also be used to aid in interpretation of other types of data. The last lines load microarray data from batch growth on ethanol compared to glucose and perform a Reporter metabolites test. These are metabolites around which significant transcriptional changes occur (for such a small model it is not super interesting though, because there are changes almost everywhere). The very last lines plot all reactions involving any of those metabolites. This can be particularly useful when working with large models.

- [readtable](https://www.mathworks.com/help/matlab/ref/readtable.html)

```MATLAB
T = readtable(filename,opts)
```
Creates a table using the import options opts. [delimitedTextImportOptions](https://www.mathworks.com/help/matlab/ref/matlab.io.text.delimitedtextimportoptions.html)

In [29]:
filename = './tutorial_data/expression.txt';
opts = delimitedTextImportOptions('VariableNames', {'openReadingFrame', 'pValue'}, ...
                                    'VariableTypes', {'char', 'double'}, ...
                                    'Delimiter', '\t', ...
                                    'Whitespace', '\b');
disp(preview(filename, opts));

    openReadingFrame    pValue 
    ________________    _______

     {'RTM1'     }      0.52257
     {'SUC4'     }            1
     {'YHR021W'  }      0.36936
     {'YOR008C'  }      0.95133
     {'YJL086C'  }      0.52477
     {'YOR387C'  }      0.90097
     {'YEL076W-C'}      0.19623
     {'YGL158W'  }      0.33086



In [30]:
reporterTable = readtable(filename, opts)

reporterTable = 6383x2 table
   <strong>openReadingFrame</strong>    <strong>pValue</strong> 
    <strong>________________</strong>    <strong>_______</strong>

     {'RTM1'     }      0.52257
     {'SUC4'     }            1
     {'YHR021W'  }      0.36936
     {'YOR008C'  }      0.95133
     {'YJL086C'  }      0.52477
     {'YOR387C'  }      0.90097
     {'YEL076W-C'}      0.19623
     {'YGL158W'  }      0.33086
     {'YOR199W'  }      0.77312
     {'YGL222C'  }      0.19623
     {'YDL163W'  }      0.61951
     {'YML084W'  }      0.21644
     {'YER044C'  }      0.73427
     {'YCR101C'  }      0.10562
     {'YKL036C'  }      0.45677
     {'YHR131W'  }       0.6355



- [reporterMetabolites](https://sysbiochalmers.github.io/RAVEN/doc/core/reporterMetabolites.html)

```MATLAB
function repMets=reporterMetabolites(model,genes,genePValues,printResults,outputFile,geneFoldChanges)
```
The Reporter Metabolites algorithm for identifying metabolites around which transcriptional changes occur

❗ Warning: Seems like it is neccesary another MATLAB add-on:

```shell
'norminv' requires Statistics and Machine Learning Toolbox.

Error in reporterMetabolites (line 78)
geneZScores=norminv(genePValues)*-1;
```

It would be very helpfull if those dependencies are more explicit in the documentation. 💡

In [31]:
repMets= reporterMetabolites(model,reporterTable.openReadingFrame, reporterTable.pValue);

[sortrows](https://www.mathworks.com/help/matlab/ref/double.sortrows.html#d124e1457202)

```MATLAB
tblB = sortrows(tblA,vars)
```
sorts a table by the elements in the variables specified by vars. For example, sortrows(tblA,{'Var1','Var2'}) first sorts the rows of tblA based on the elements in Var1, then by the elements in Var2.

- If tblA is a table and it has row names, then vars can include the row names.
- If tblA is a timetable, then vars can include the row times.

In [32]:
% make a table from the reporter metabolites, next filter them by their p-value and select the first 10 values
sortedTable = sortrows(table(repMets.mets, repMets.metPValues, 'variableNames', {'metaboliteID', 'pValue'}), 'pValue');
sortedTableTop10 = sortedTable(1:10,:);
fprintf('Top 10 reporter metabolites (by p-value):\n\n');
disp(sortedTableTop10);

Top 10 reporter metabolites (by p-value):

    metaboliteID     pValue 
    ____________    ________

    {'F6P_c'  }     0.044932
    {'E4P_c'  }     0.067485
    {'S7P_c'  }     0.067485
    {'P6G_c'  }     0.068213
    {'ACCOA_c'}     0.089614
    {'COA_c'  }     0.089614
    {'CI_m'   }      0.10217
    {'RU5P_c' }      0.11807
    {'GLC_c'  }      0.13644
    {'NADP_c' }      0.14617



In [33]:
selectedMetsIndexes = getIndexes(model, sortedTableTop10.metaboliteID, 'mets');
% in which reactions those metabolities participate
[ ~ , selectedReactionsIndexes] = find(model.S(selectedMetsIndexes, :));

[trimPathway](https://sysbiochalmers.github.io/RAVEN/doc/pathway/trimPathway.html)

```MATLAB
function pathway=trimPathway(pathway, rxnsToKeep, deleteUnconnectedMets)
```

Delete reactions from a pathway object

In [34]:
updatedPathway= trimPathway(pathway, model.rxns(selectedReactionsIndexes), true);
% notMapped=drawMap(title,pathway,modelA,conditionA,conditionB,modelB,filename,cutOff,supressOpen)
drawMap('Reactions involving the top 10 Reporter Metabolites', ...
    updatedPathway, model, ones(numel(model.rxns),1), zeros(numel(model.rxns),1), [], ...
    './output/tutorial_3_mapRM.pdf', tolerance);

File saved as ./output/tutorial_3_mapRM.pdf


In [36]:
clear